 # MongoDB Vector Search Workshop: WoW Game Data Demo



 **주제:** Keyword Search → Naive Vector Search → Contextual Embedding → Reranking



 이 노트북에서는 다음 내용을 다룹니다:

 1. **Keyword Search:** MongoDB Atlas Search를 이용한 전통적인 텍스트 검색

 2. **Naive Vector Search:** 단순 임베딩을 이용한 벡터 검색의 한계

 3. **Contextual Embedding:** 메타데이터를 포함한 향상된 벡터 검색

 4. **Voyage AI Reranking:** 검색 결과 재정렬로 정확도 향상



 ---

 **시나리오:** 레벨 5 오크 플레이어에게 적합한 무기 추천하기



 **문제:** Naive Vector Search는 레벨 99 전설 무기를 추천 (착용 불가)

 **해결:** Contextual Embedding으로 플레이어 레벨과 진영 정보를 반영

 ## 환경 설정 및 라이브러리 임포트



 필요한 라이브러리들을 임포트하고 API 키와 MongoDB URI를 설정합니다.

In [25]:
!pip install requests pymongo voyageai dnspython

import os
import time
import requests
from pymongo import MongoClient
from pymongo.operations import SearchIndexModel
import voyageai


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


 ## API 키 및 데이터베이스 설정



 **중요:** 아래 환경변수를 설정해주세요:

 - `VOYAGE_API_KEY`: Voyage AI API 키

 - `MONGODB_URI`: MongoDB Atlas 연결 문자열

In [ ]:
# ==========================================
# [설정] API 키와 MONOGDB URI 설정
# ==========================================
MONGODB_URI = ""
VOYAGE_API_KEY = ""  
      
client = MongoClient(MONGODB_URI)
db = client["wow_workshop_db"]
collection = db["game_data"]
vo = voyageai.Client(api_key=VOYAGE_API_KEY)  # ✅ API 키 전달

try:
    client.admin.command('ping')
    print("✅ MongoDB 연결 성공!")
    
    db = client["wow_workshop_db"]
    collection = db["game_data"]
    vo = voyageai.Client(api_key=VOYAGE_API_KEY)
    
    print("✅ Clients initialized.")
    
except Exception as e:
    print(f"❌ 연결 실패: {e}")

✅ MongoDB 연결 성공!
✅ Clients initialized.


 ## 유틸리티 함수 정의



 검색 결과를 보기 좋게 출력하는 헬퍼 함수입니다.

In [34]:
def print_result(title, docs):
    """검색 결과를 포맷팅하여 출력하는 함수"""
    print(f"\n--- [ {title} ] ---")
    if not docs:
        print("   (No results found)")
        return
    for i, d in enumerate(docs):
        name = d.get('name', d.get('title', 'Unknown'))
        desc = d.get('description', d.get('content', ''))
        # None 체크 추가
        if desc:
            desc = desc[:80] + "..." if len(desc) > 80 else desc
        else:
            desc = "No description available"
        level = d.get('req_level', 'N/A')
        score = d.get('score', 0)
        print(f"{i+1}. {name} (Lv.{level}) | Score: {score:.4f}")
        print(f"   Desc: {desc}")
    print("---------------------------------")


def wait_for_index(index_name):
    print(f"   Waiting for index '{index_name}' to be READY...")
    while True:
        indices = list(collection.list_search_indexes(index_name))
        if indices and indices[0].get('queryable'):
            print(f"✅ Index '{index_name}' is READY!")
            break
        time.sleep(1)

 ## Step 1: 데이터 로드



 GitHub에서 WoW 게임 데이터셋을 다운로드하고 MongoDB에 삽입합니다.



 데이터셋에는 아이템, 퀘스트, NPC 정보가 포함되어 있습니다.

In [27]:
# 데이터 로드 (GitHub에서)
DATASET_URL = "https://raw.githubusercontent.com/Jackkkie/mongodb-kor-ps-gen-ai-workshop/main/dataset.json"

print("⏳ Downloading dataset from GitHub...")
response = requests.get(DATASET_URL)
data = response.json()

# 기존 데이터 삭제 후 새로 삽입
collection.delete_many({})
collection.insert_many(data)
print(f"✅ Successfully loaded {len(data)} documents into MongoDB.")

⏳ Downloading dataset from GitHub...
✅ Successfully loaded 22 documents into MongoDB.


 ## Step 2: Keyword Search (전통적인 텍스트 검색)



 MongoDB Atlas Search를 사용하여 키워드 기반 검색을 수행합니다.



 **장점:** 빠르고 간단함

 **단점:** 의미적 유사성을 이해하지 못함

In [35]:
print("\n🔍 [Step 1] Setting up Keyword Search Index...")
index_name = "default_text_index"
model = SearchIndexModel(
    definition={"mappings": {"dynamic": True}}, 
    name=index_name
)

try:
    collection.create_search_index(model=model)
except Exception:
    print(f"   (Index '{index_name}' creation request sent or already exists)")

wait_for_index(index_name)


🔍 [Step 1] Setting up Keyword Search Index...
   Waiting for index 'default_text_index' to be READY...
✅ Index 'default_text_index' is READY!


In [37]:
# Keyword Search 실행
# query_term = "Strong Weapon"
# pipeline = [
#     {
#         "$search": {
#             "index": index_name, 
#             "text": {
#                 "query": query_term, 
#                 "path": {"wildcard": "*"}
#             }
#         }
#     },
#     {"$limit": 3},
#     {
#         "$project": {
#             "name": 1, 
#             "description": 1, 
#             "req_level": 1, 
#             "score": {"$meta": "searchScore"}
#         }
#     }
# ]

# results = list(collection.aggregate(pipeline))
# print_result(f"Keyword Search: '{query_term}'", results)
# %%
# 데이터 샘플 확인
sample = collection.find_one({"type": "item"})
if sample:
    print("아이템 데이터 구조:")
    for key, value in sample.items():
        print(f"  {key}: {value[:50] if isinstance(value, str) and len(value) > 50 else value}")
else:
    print("아이템 데이터가 없습니다.")

# 전체 데이터 타입 확인
types = collection.distinct("type")
print(f"\n데이터 타입들: {types}")

# 각 타입별 개수
for t in types:
    count = collection.count_documents({"type": t})
    print(f"  {t}: {count}개")


아이템 데이터 구조:
  _id: 692648e9e7db612c035d4430
  doc_id: ITEM-001
  type: item
  name: 수습생의 낡은 도끼 (Apprentice's Rusty Axe)
  req_level: 1
  faction: Horde
  description: 듀로타의 어린 오크들이 훈련용으로 사용하는 무딘 도끼입니다. 멧돼지 가죽을 자르기에 적합합
  image_description: A chipped, rusty iron axe with a wooden handle wra
  image_url: https://raw.githubusercontent.com/Jackkkie/mongodb
  context_tag: Tutorial, Beginner, Level 1-5, Melee Weapon

데이터 타입들: ['item', 'location', 'lore', 'monster', 'npc', 'quest', 'user_save']
  item: 9개
  location: 2개
  lore: 2개
  monster: 2개
  npc: 2개
  quest: 3개
  user_save: 2개


 ## Step 3: Naive Vector Search



 Voyage AI를 사용하여 문서의 설명(description)만으로 임베딩을 생성합니다.



 **문제점:** 메타데이터(레벨, 진영 등)를 고려하지 않아 부적절한 추천이 발생할 수 있습니다.

In [ ]:
print("\n🤖 [Step 2] Generating Naive Embeddings (Voyage-3)...")

# 아이템 문서들 가져오기
docs = list(collection.find({"type": "item"}))

# 설명 텍스트만 추출
texts = [d.get("description", "") for d in docs]

# Voyage Embedding 생성 (배치 처리)
vectors = vo.embed(texts, model="voyage-3", input_type="document").embeddings

# MongoDB에 임베딩 업데이트
for doc, vec in zip(docs, vectors):
    collection.update_one(
        {"_id": doc["_id"]}, 
        {"$set": {"naive_embedding": vec}}
    )


In [ ]:
# Vector Index 생성
print("   Creating Vector Index...")
vector_index_name = "naive_vector_index"
vector_model = SearchIndexModel(
    definition={
        "fields": [
            {
                "type": "vector", 
                "path": "naive_embedding", 
                "numDimensions": 1024, 
                "similarity": "cosine"
            }
        ]
    },
    name=vector_index_name,
    type="vectorSearch"
)

try: 
    collection.create_search_index(model=vector_model)
except: 
    pass

print("   Waiting for Vector Index (approx 10-30s)...")
time.sleep(15)  # Atlas Vector Index는 생성에 시간이 좀 걸립니다.


In [ ]:
# Naive Vector Search 실행
user_query = "Recommend a strong weapon"
query_vec = vo.embed([user_query], model="voyage-3", input_type="query").embeddings[0]

pipeline = [
    {
        "$vectorSearch": {
            "index": vector_index_name, 
            "path": "naive_embedding", 
            "queryVector": query_vec, 
            "limit": 3, 
            "numCandidates": 50
        }
    },
    {
        "$project": {
            "name": 1, 
            "description": 1, 
            "req_level": 1, 
            "score": {"$meta": "vectorSearchScore"}
        }
    }
]

results = list(collection.aggregate(pipeline))
print_result(f"Naive Vector Search: '{user_query}'", results)
print("👉 문제점: 레벨 99짜리 서리한(Frostmourne)이 추천됨 (스포일러/착용불가)")


 ## Step 4: Contextual Embedding (해결책)



 메타데이터를 텍스트에 포함시켜 임베딩을 생성합니다.



 **개선점:**

 - 레벨 정보 포함

 - 진영 정보 포함

 - 아이템 타입 명시



 이를 통해 플레이어 상황에 맞는 적절한 추천이 가능합니다.

In [ ]:
print("\n🧠 [Step 3] Generating Contextual Embeddings...")

docs = list(collection.find({"type": "item"}))
context_texts = []

# 메타데이터를 텍스트로 주입
for d in docs:
    txt = f"""Target Level: {d.get('req_level')}
Faction: {d.get('faction')}
Item: {d.get('name')}
Desc: {d.get('description')}"""
    context_texts.append(txt)

# Voyage Embedding 생성 (메타데이터 포함)
vectors = vo.embed(context_texts, model="voyage-3", input_type="document").embeddings

# MongoDB 업데이트
for doc, vec, txt in zip(docs, vectors, context_texts):
    collection.update_one(
        {"_id": doc["_id"]}, 
        {
            "$set": {
                "contextual_embedding": vec, 
                "contextual_text": txt
            }
        }
    )


In [ ]:
# Contextual Vector Index 생성
ctx_index_name = "contextual_vector_index"
ctx_model = SearchIndexModel(
    definition={
        "fields": [
            {
                "type": "vector", 
                "path": "contextual_embedding", 
                "numDimensions": 1024, 
                "similarity": "cosine"
            }
        ]
    },
    name=ctx_index_name,
    type="vectorSearch"
)

try: 
    collection.create_search_index(model=ctx_model)
except: 
    pass

print("   Waiting for Context Index (approx 10-30s)...")
time.sleep(15)


In [ ]:
# Contextual Search 실행
# 유저 상태: Level 5, Horde
player_context = """Player Level: 5
Player Faction: Horde
Intent: Need equipment update
"""

final_query = player_context + "Query: " + user_query

print(f"📝 Enhanced Query:\n{final_query}")

# 컨텍스트가 포함된 쿼리로 임베딩 생성
query_vec = vo.embed([final_query], model="voyage-3", input_type="query").embeddings[0]

pipeline = [
    {
        "$vectorSearch": {
            "index": ctx_index_name, 
            "path": "contextual_embedding", 
            "queryVector": query_vec, 
            "limit": 5, 
            "numCandidates": 50
        }
    },
    {
        "$project": {
            "name": 1, 
            "description": 1, 
            "req_level": 1, 
            "score": {"$meta": "vectorSearchScore"}
        }
    }
]

results = list(collection.aggregate(pipeline))
print_result("Contextual Search Result", results)
print("👉 해결: 이제 레벨 5 오크에게 맞는 '수습생의 도끼'가 상단에 뜸.")


 ## Step 5: Voyage AI Reranking



 검색된 결과를 Voyage AI의 Reranking 모델로 재정렬하여

 더욱 정확한 관련성 순위를 매깁니다.



 **장점:**

 - 초기 검색 결과의 정확도 향상

 - 쿼리와 문서 간의 깊은 의미적 유사성 파악

In [ ]:
print("\n🏆 [Step 4] Reranking with Voyage AI...")

# 검색된 상위 결과들 가져오기
documents_to_rerank = [doc['description'] for doc in results]
doc_names = [doc['name'] for doc in results]

# Voyage AI Reranking
reranking = vo.rerank(
    query=user_query,
    documents=documents_to_rerank,
    model="rerank-2",
    top_k=3
)

# Reranking 결과 출력
print(f"Query for Rerank: '{user_query}'")
for r in reranking.results:
    print(f"Rank {r.index + 1}: {doc_names[r.index]} (Score: {r.relevance_score:.4f})")


 ## 결론



 이 데모에서 우리는 다음을 배웠습니다:



 1. **Keyword Search**는 빠르지만 의미를 이해하지 못함

 2. **Naive Vector Search**는 의미는 이해하지만 컨텍스트를 놓침

 3. **Contextual Embedding**으로 메타데이터를 활용한 정확한 검색 가능

 4. **Reranking**으로 최종 결과의 품질 향상



 **핵심 교훈:** RAG 시스템에서는 단순히 임베딩만이 아니라,

 메타데이터와 컨텍스트를 어떻게 활용하느냐가 성능의 핵심입니다.

In [ ]:
print("\n✅ Demo Complete.")